In [1]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

DATA_DIR = DIR/'data/sl/'

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
path = DATA_DIR/'/home/pablo/dev23/data_sets/sl.zip'
gk.list_feed(path)

,file_name,file_size
0,stop_times.txt,132092280
1,calendar.txt,27620
2,agency.txt,1278
3,stops.txt,1434719
4,calendar_dates.txt,676779
5,feed_info.txt,144
6,routes.txt,30622
7,attributions.txt,2279559
8,trips.txt,4804496
9,transfers.txt,1287404


In [3]:
feed = gk.read_feed(path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,"[Upplands lokaltrafik, Kommuner i Stockholms L..."
1,timezone,Europe/Stockholm
2,start_date,20240212
3,end_date,20241230
4,num_routes,610
5,num_trips,77417
6,num_stops,21568
7,num_shapes,3211
8,sample_date,20240215
9,num_routes_active_on_sample_date,575


In [4]:
feed.validate()
feed.drop_invalid_columns()
feed.drop_zombies()

feed_info: dropping invalid column feed_id
stops: dropping invalid column platform_code
transfers: dropping invalid column from_trip_id
transfers: dropping invalid column to_trip_id


In [5]:
# Append shape_dist_traveled column to stop times
display(feed.stop_times.head().T)
feed = feed.append_dist_to_stop_times()
display(feed.stop_times.head().T)

,0,1,2,3,4
trip_id,14010000635912911,14010000635912911,14010000635912911,14010000635912911,14010000635912911
arrival_time,05:53:00,05:53:56,05:54:23,05:54:57,05:55:45
departure_time,05:53:00,05:53:56,05:54:23,05:54:57,05:55:45
stop_id,9022001010028003,9022001010052003,9022001010054002,9022001010056002,9022001010058002
stop_sequence,1,2,3,4,5
stop_headsign,Stora Essingen,Stora Essingen,Stora Essingen,Stora Essingen,Stora Essingen
pickup_type,3,3,3,3,3
drop_off_type,1,3,3,3,3
shape_dist_traveled,0.0,398.26,570.67,808.54,1107.9
timepoint,1,0,0,0,0


,0,1,2,3,4
trip_id,14010000331224227,14010000331224227,14010000331224227,14010000331224227,14010000331224227
arrival_time,06:39:00,06:40:00,06:42:00,06:44:00,06:45:00
departure_time,06:39:00,06:40:00,06:42:00,06:44:00,06:45:00
stop_id,9022001004201001,9022001004411001,9022001004421001,9022001004431001,9022001004441001
stop_sequence,1,2,3,4,5
stop_headsign,Nockeby,Nockeby,Nockeby,Nockeby,Nockeby
pickup_type,3,3,3,3,3
drop_off_type,1,3,3,3,3
timepoint,1,1,1,1,1
shape_dist_traveled,0.0,0.864388,1.329668,2.189957,2.741705


In [6]:
week = feed.get_first_week()
dates = [week[4], week[6]]  # First Friday and Sunday

In [7]:
# Compute feed time series
trip_stats = feed.compute_trip_stats()
trip_stats.head().T
fts = feed.compute_feed_time_series(trip_stats, dates, freq='6h')
fts

/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/trips.py:329: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  h[["start_time", "end_time"]] = h[["start_time", "end_time"]].applymap(
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/routes.py:529: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rng = pd.date_range(start, end, freq=freq)
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/routes.py:358: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tss[["start_index", "end_index"]] = tss[["start_time", "end_time"]].applymap(F)
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/helpers.py:400: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  if f.empty or pd.tseries.frequencies.to_offset(
/home/pablo/anaconda3/envs/geo_env/li

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2024-02-16 00:00:00,2289.0,2969.0,2951.0,42154.185621,1339.716667,31.465001
2024-02-16 06:00:00,12186.0,12375.0,13030.0,176824.098790,6738.116667,26.242362
2024-02-16 12:00:00,12408.0,12850.0,13678.0,181212.670979,7181.166667,25.234433
2024-02-16 18:00:00,8216.0,7243.0,8543.0,112408.031917,3964.733333,28.351978
2024-02-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-17 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-17 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-17 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-18 00:00:00,1048.0,1242.0,1235.0,20864.200275,670.933333,31.097278


In [8]:
gk.downsample(fts, freq='12h')

/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/helpers.py:400: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  if f.empty or pd.tseries.frequencies.to_offset(
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/helpers.py:427: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  num_trips = f.groupby(pd.Grouper(freq=freq)).apply(agg_num_trips)
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/helpers.py:433: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  f[ind].resample(freq).agg(lambda x: x.sum(min_count=1))
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/helpers.py:451: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  result.index.freq = freq


indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2024-02-16 00:00:00,14475.0,15344.0,15319.0,218978.284411,8077.833333,27.108542
2024-02-16 12:00:00,20624.0,20093.0,20951.0,293620.702895,11145.900000,26.343382
2024-02-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-17 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-18 00:00:00,6494.0,7149.0,7118.0,102993.386566,3523.683333,29.228900
2024-02-18 12:00:00,13705.0,13367.0,14009.0,195009.526835,7126.316667,27.364701


In [9]:
# Compute feed stats for first week
feed_stats = feed.compute_feed_stats(trip_stats, week)
feed_stats

/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/miscellany.py:262: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ts[["start_time", "end_time"]] = ts[["start_time", "end_time"]].applymap(
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/miscellany.py:322: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  stats[times] = stats[times].applymap(
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/miscellany.py:262: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ts[["start_time", "end_time"]] = ts[["start_time", "end_time"]].applymap(
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/miscellany.py:322: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  stats[times] = stats[times].applymap(
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit

,num_stops,num_routes,num_trips,num_trip_starts,num_trip_ends,service_distance,service_duration,service_speed,peak_num_trips,peak_start_time,peak_end_time,date
0,11787,574,34644,34644,33431,501028.682063,18856.462500,26.570661,1610,07:51:00,07:51:15,20240212
0,11787,576,34659,34659,33446,501208.768805,18864.979167,26.568212,1611,07:51:00,07:51:15,20240213
0,11787,574,34663,34663,33450,501264.142286,18868.095833,26.566758,1611,07:51:00,07:51:15,20240214
0,11787,575,34661,34661,33448,501346.539147,18869.812500,26.568708,1611,07:51:00,07:51:15,20240215
0,11787,578,35437,35437,33454,512598.987307,19224.395833,26.663984,1608,07:51:00,07:51:15,20240216
0,11787,415,21877,21877,19902,315664.124694,11311.237500,27.907126,762,15:05:15,15:05:30,20240217
0,11787,415,20516,20516,19312,298002.913401,10650.583333,27.979962,743,15:40:45,15:41:00,20240218


In [10]:
# Compute route time series
rts = feed.compute_route_time_series(trip_stats, dates, freq='12H')
rts

/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/routes.py:529: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rng = pd.date_range(start, end, freq=freq)
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/routes.py:358: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tss[["start_index", "end_index"]] = tss[["start_time", "end_time"]].applymap(F)
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/helpers.py:400: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  if f.empty or pd.tseries.frequencies.to_offset(
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/helpers.py:427: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  num_trips = f.groupby(pd.Grouper(freq=freq)).apply(agg_num_trips)
/home/pablo/anaconda3/env

indicator              num_trip_ends                                    \
route_id            9011001000100000 9011001000200000 9011001000300000   
datetime                                                                 
2024-02-16 00:00:00            101.0             95.0            100.0   
2024-02-16 12:00:00            162.0            146.0            145.0   
2024-02-17 00:00:00              NaN              NaN              NaN   
2024-02-17 12:00:00              NaN              NaN              NaN   
2024-02-18 00:00:00             56.0             60.0             63.0   
2024-02-18 12:00:00            125.0            122.0            122.0   

indicator                                                               \
route_id            9011001000400000 9011001000600000 9011001000700000   
datetime                                                                 
2024-02-16 00:00:00            115.0             81.0             84.0   
2024-02-16 12:00:00            198.0            135.0            158.0   
2024-02-17 00:00:00              NaN              NaN              NaN   
2024-02-17 12:00:00              NaN              NaN              NaN   
2024-02-18 00:00:00             65.0             38.0             60.0   
2024-02-18 12:00:00            153.0             96.0            159.0   

indicator                                                               \
route_id            9011001001000000 9011001001100000 9011001001200000   
datetime                                                                 
2024-02-16 00:00:00            117.0            118.0             89.0   
2024-02-16 12:00:00            154.0            153.0            138.0   
2024-02-17 00:00:00              NaN              NaN              NaN   
2024-02-17 12:00:00              NaN              NaN              NaN   
2024-02-18 00:00:00             65.0             64.0             36.0   
2024-02-18 12:00:00            134.0            133.0             70.0   

indicator                             ...    service_speed                   \
route_id            9011001001300000  ... 9011008002000000 9011008002100000   
datetime                              ...                                     
2024-02-16 00:00:00            143.0  ...        18.633186        17.627850   
2024-02-16 12:00:00            191.0  ...        19.058579        16.698539   
2024-02-17 00:00:00              NaN  ...              NaN              NaN   
2024-02-17 12:00:00              NaN  ...              NaN              NaN   
2024-02-18 00:00:00             60.0  ...        18.894262        15.889901   
2024-02-18 12:00:00            138.0  ...        17.507598        16.369555   

indicator                                                               \
route_id            9011008002200000 9011008002300000 9011008002500000   
datetime                                                                 
2024-02-16 00:00:00        14.500351        11.487748         0.000000   
2024-02-16 12:00:00        14.496769        11.041190        14.667866   
2024-02-17 00:00:00              NaN              NaN              NaN   
2024-02-17 12:00:00              NaN              NaN              NaN   
2024-02-18 00:00:00         0.000000        10.156436         0.000000   
2024-02-18 12:00:00        14.497127        11.280067        12.834382   

indicator                                                               \
route_id            9011008002900000 9011008003000000 9011008003100000   
datetime                                                                 
2024-02-16 00:00:00        10.653874        11.449547        16.831950   
2024-02-16 12:00:00        10.653874        11.449547        17.785629   
2024-02-17 00:00:00              NaN              NaN              NaN   
2024-02-17 12:00:00              NaN              NaN              NaN   
2024-02-18 00:00:00        10.653874        11.449547        18.938285   
2024-02-18 12:00:00        1

In [11]:
# Slice time series
inds = ['service_distance', 'service_duration', 'service_speed']
rids = ['9011001000200000', '9011001000300000'] # [478, 480, 481, 607, 608]
rts.loc[:, (inds, rids)]

indicator           service_distance                  service_duration  \
route_id            9011001000200000 9011001000300000 9011001000200000   
datetime                                                                 
2024-02-16 00:00:00       786.248378      1000.015205        61.200000   
2024-02-16 12:00:00      1117.984908      1430.898209        93.033333   
2024-02-17 00:00:00              NaN              NaN              NaN   
2024-02-17 12:00:00              NaN              NaN              NaN   
2024-02-18 00:00:00       501.040518       668.344919        30.466667   
2024-02-18 12:00:00       944.722635      1207.466974        66.750000   

indicator                               service_speed                   
route_id            9011001000300000 9011001000200000 9011001000300000  
datetime                                                                
2024-02-16 00:00:00        70.816667        12.847196        14.121184  
2024-02-16 12:00:00       106.616667        12.017036        13.420962  
2024-02-17 00:00:00              NaN              NaN              NaN  
2024-02-17 12:00:00              NaN              NaN              NaN  
2024-02-18 00:00:00        40.050000        16.445531        16.687763  
2024-02-18 12:00:00        80.866667        14.153148        14.931578

In [12]:
# Slice again by cross-section
rts.xs(rids[0], axis="columns", level=1)

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2024-02-16 00:00:00,95.0,104.0,104.0,786.248378,61.200000,12.847196
2024-02-16 12:00:00,146.0,141.0,150.0,1117.984908,93.033333,12.017036
2024-02-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-17 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-18 00:00:00,60.0,66.0,65.0,501.040518,30.466667,16.445531
2024-02-18 12:00:00,122.0,120.0,126.0,944.722635,66.750000,14.153148


In [13]:
# Compute trip locations for every hour
rng = pd.date_range('2/16/2024', periods=24, freq='h')
times = [t.strftime('%H:%M:%S') for t in rng]
loc = feed.locate_trips(dates[0], times)
loc.head()

trip_id      time  rel_dist  \
shape_id                                                          
1014010000001321141 1365  14010000576612254  20:00:00  0.000000   
                    1366  14010000576612685  21:00:00  0.086790   
                    1467  14010000586114939  13:00:00  0.780027   
                    1468  14010000586115464  14:00:00  0.780027   
                    4697  14010000630966220  09:00:00  0.502373   

                                  route_id  direction_id             shape_id  \
shape_id                                                                        
1014010000001321141 1365  9011001011400000             1  1014010000001321141   
                    1366  9011001011400000             1  1014010000001321141   
                    1467  9011001011400000             1  1014010000001321141   
                    1468  9011001011400000             1  1014010000001321141   
                    4697  9011001011400000             1  1014010000001321141   

                                lon        lat  
shape_id                                        
1014010000001321141 1365  17.980415  59.332968  
                    1366  17.978586  59.335035  
                    1467  17.977896  59.338987  
                    1468  17.977896  59.338987  
                    4697  17.987259  59.340102

In [14]:
# Build a route timetable
route_id = feed.routes['route_id'].iat[0]
feed.build_route_timetable(route_id, dates).T

/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/routes.py:677: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  f["dt"] = f["departure_time"].fillna(method="ffill")
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/routes.py:678: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  f["min_dt"] = f.groupby("trip_id")["dt"].transform(min)
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_kit/routes.py:677: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  f["dt"] = f["departure_time"].fillna(method="ffill")
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/gtfs_ki

,20239,20240,20241,20242,20243,20244,20245,20246,20247,20248,...,28059,28060,28061,28062,28063,28064,28065,28066,28067,28068
route_id,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000,...,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000,9011001000100000
service_id,19,19,19,19,19,19,19,19,19,19,...,24,24,24,24,24,24,24,24,24,24
trip_id,14010000641551312,14010000641551312,14010000641551312,14010000641551312,14010000641551312,14010000641551312,14010000641551312,14010000641551312,14010000641551312,14010000641551312,...,14010000641585892,14010000641585892,14010000641585892,14010000641585892,14010000641585892,14010000641585892,14010000641585892,14010000641585892,14010000641585892,14010000641585892
trip_headsign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
direction_id,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
shape_id,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,...,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820,1014010000638766820
arrival_time,05:00:00,05:00:40,05:01:14,05:03:00,05:04:02,05:04:41,05:05:22,05:06:00,05:07:40,05:09:00,...,25:01:00,25:01:53,25:03:10,25:04:00,25:05:59,25:07:25,25:08:07,25:09:13,25:10:03,25:12:00
departure_time,05:00:00,05:00:40,05:01:14,05:03:00,05:04:02,05:04:41,05:05:22,05:06:00,05:07:40,05:09:00,...,25:01:00,25:01:53,25:03:10,25:04:00,25:05:59,25:07:25,25:08:07,25:09:13,25:10:03,25:12:00
stop_id,9022001010736001,9022001010733002,9022001010455002,9022001010453002,9022001010451002,9022001010449002,9022001010462001,9022001010645004,9022001010421002,9022001010369005,...,9022001010045002,9022001010153001,9022001010155001,9022001010062002,9022001010060001,9022001010058001,9022001010056001,9022001010054001,9022001010052002,9022001010028001
stop_sequence,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30


In [15]:
rids = feed.routes.route_id.loc[2:4]
feed.map_routes(rids)

ValueError: must have exactly one of create/read/write/append mode

In [ ]:
# Compute screen line counts

#lines = gp.read_file(path)
#display(lines)
#feed.compute_screen_line_counts(lines, dates)

In [ ]:
# Map routes
#rids = feed.routes.route_id.loc[2:4]
#feed.map_routes(rids, include_stops=True)